In [1]:
# Due to this problem:https://courses.edx.org/courses/course-v1:IBM+DS0720EN+2T2021/discussion/forum/course/threads/61cd068fb38ee505ec0366b2
# I had to write the code on my Jupyter Notebook

!pip install dash==1.19.0

!pip install jupyter_dash

!pip install --upgrade plotly

     |████████████████████████████████| 75 kB 5.9 MB/s  eta 0:00:01
     |████████████████████████████████| 95 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 1.0 MB 18.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 63.5 MB/s eta 0:00:01
     |████████████████████████████████| 188 kB 61.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 31.0 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 80.4 MB/s eta 0:00:01
  Created wheel for dash: filename=dash-1.19.0-py3-none-any.whl size=84011 sha256=e1a388f0e17391fa15bc3a95270051adf09386f11eabdec63318c54b8ef99df6
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/e7/27/2f/a9f9bf858843b5365bcb311a80f0b2379737caf4ec50b2ae3a
  Created wheel for dash-core-components: filename=dash_core_components-1.15.0-py3-none-any.whl size=3527011 sha256=f7ce1c87b34118930df74002a62790b709c3b200a5f6a18fa111b1ffefa833d2
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/e3/e6/0e/759ca580c568aad2

In [2]:
import pandas as pd 
import dash 
import dash_html_components as html 
import dash_core_components as dcc 
from dash.dependencies import Input, Output 
from jupyter_dash import JupyterDash 
import plotly.graph_objects as go

In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown', options=[
                                {'label': 'All Sites', 'value': '5'},
                                {'label': 'CCAFS LC-40','value':'1'},                           
                                {'label':'VAFB SLC-4E','value':'2'},
                                {'label':'KSC LC-39A','value':'3'},
                                {'label':'CCAFS SLC-40','value':'4'}],
                                value=5,
                                placeholder='Select a Launch Site here',
                                searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',min=0,max=10000,
                                step=1000,value=[min_payload,max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])
x=['CCAFS LC-40','VAFB SLC-4E','KSC LC-39A','CCAFS SLC-40']
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart',component_property='figure'),
                Input(component_id='site-dropdown',component_property='value'))
def get_graph(value):
    if value=='1' or value=='2' or value=='3' or value=='4':
        my_df=spacex_df[spacex_df['Launch Site']==x[int(value)-1]]
        fig=px.pie(my_df,names='class',title='Success-Failure for Site {}'.format(x[int(value)-1]))
        return fig
    else:
        xfig=px.pie(spacex_df,values='class',names='Launch Site',title='Total Success Launches for All Sites {}'.format(value))
        return xfig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                Input(component_id='site-dropdown',component_property='value'))
def get_graph(value):
    if value=='1' or value=='2' or value=='3' or value=='4':
        my_df=spacex_df[spacex_df['Launch Site']==x[int(value)-1]]
        fig=px.scatter(my_df,x='Payload Mass (kg)',y='class',color="Booster Version Category".format(value))
        return fig
    else:
        xfig=px.scatter(spacex_df,x='Payload Mass (kg)',y='class',color="Booster Version Category".format(value))
        return xfig
    

# Run the app
if __name__ == '__main__': 
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, #debug=True, 
                   dev_tools_hot_reload =True, threaded=True)
    
    
#Use the URL in browser for results: http://localhost:8090